In [1]:
import polars as pl

In [2]:
df = pl.scan_parquet("data/traces.parquet")

### Objective is to construct balance_diffs cryo dataset

there are two ways that a transaction can pay the block builder. gas priority fee, or the COINBASE opcode (which sends ETH directly to the block fee recipient). COINBASE is advanced mode, used mainly by MEV searchers. vanilla tx data will only reveal the priority fee, not the COINBASE operations. so you need something like the parity stateDiff traces. these traces will show all of the 1) balances, 2) contract bytecodes, 3) nonces, and 4) storage slots that changed for each transaction - Storm

In [3]:
df.select("block_number").min().collect().item()

19999999

In [4]:
df.select("block_number").max().collect().item()

20025000

In [5]:
df.head(10).collect()

from,to,call_type,gas,input,init,value,author,reward_type,block_hash,block_number,address,code,gas_used,output,subtraces,trace_address,transaction_hash,transaction_position,type,error
str,str,str,str,str,str,str,str,str,str,u64,str,str,f64,str,u64,str,str,u64,str,str
"""0xae2fc483527b8ef99eb5d9b44875…","""0x6b75d8af000000e20b7a7ddf000b…","""call""","""0x0a5f3d""","""0x2b669bdc7dfd19b75b023e28bbb8…",null,"""0xff""",null,null,"""0xb390d63aac03bbef75de888d16bd…",19999999,null,null,475819.0,"""0x""",6,"""0x0000000000000000""","""0x5a8f63301f5a9b267c63b4e97090…",0,"""call""",null
"""0x6b75d8af000000e20b7a7ddf000b…","""0x9bdc7dfd19b75b023e28bbb8e197…","""call""","""0x07f6ca""","""0x128acb0800000000000000000000…",null,"""0x00""",null,null,"""0xb390d63aac03bbef75de888d16bd…",19999999,null,null,49428.0,"""0xffffffffffffffffffffffffffff…",4,"""0x0100000000000000000000000000…","""0x5a8f63301f5a9b267c63b4e97090…",0,"""call""",null
"""0x9bdc7dfd19b75b023e28bbb8e197…","""0x52498f8d9791736f1d6398fe95ba…","""call""","""0x07812c""","""0xa9059cbb00000000000000000000…",null,"""0x00""",null,null,"""0xb390d63aac03bbef75de888d16bd…",19999999,null,null,10772.0,"""0x0000000000000000000000000000…",1,"""0x0200000000000000000000000000…","""0x5a8f63301f5a9b267c63b4e97090…",0,"""call""",null
"""0x52498f8d9791736f1d6398fe95ba…","""0x903bfa4323cc7b1acee9965e1966…","""delegatecall""","""0x0761ca""","""0xa9059cbb00000000000000000000…",null,"""0x00""",null,null,"""0xb390d63aac03bbef75de888d16bd…",19999999,null,null,10376.0,"""0x0000000000000000000000000000…",0,"""0x0300000000000000000000000000…","""0x5a8f63301f5a9b267c63b4e97090…",0,"""call""",null
"""0x9bdc7dfd19b75b023e28bbb8e197…","""0xc02aaa39b223fe8d0a0e5c4f27ea…","""staticcall""","""0x07545a""","""0x70a0823100000000000000000000…",null,"""0x00""",null,null,"""0xb390d63aac03bbef75de888d16bd…",19999999,null,null,534.0,"""0x0000000000000000000000000000…",0,"""0x0200000000000000000000000000…","""0x5a8f63301f5a9b267c63b4e97090…",0,"""call""",null
"""0x9bdc7dfd19b75b023e28bbb8e197…","""0x6b75d8af000000e20b7a7ddf000b…","""call""","""0x074f46""","""0xfa461e33ffffffffffffffffffff…",null,"""0x00""",null,null,"""0xb390d63aac03bbef75de888d16bd…",19999999,null,null,9274.0,"""0x""",1,"""0x0200000000000000000000000000…","""0x5a8f63301f5a9b267c63b4e97090…",0,"""call""",null
"""0x6b75d8af000000e20b7a7ddf000b…","""0xc02aaa39b223fe8d0a0e5c4f27ea…","""call""","""0x073074""","""0xa9059cbb00000000000000000000…",null,"""0x00""",null,null,"""0xb390d63aac03bbef75de888d16bd…",19999999,null,null,8862.0,"""0x0000000000000000000000000000…",0,"""0x0300000000000000000000000000…","""0x5a8f63301f5a9b267c63b4e97090…",0,"""call""",null
"""0x9bdc7dfd19b75b023e28bbb8e197…","""0xc02aaa39b223fe8d0a0e5c4f27ea…","""staticcall""","""0x072925""","""0x70a0823100000000000000000000…",null,"""0x00""",null,null,"""0xb390d63aac03bbef75de888d16bd…",19999999,null,null,534.0,"""0x0000000000000000000000000000…",0,"""0x0200000000000000000000000000…","""0x5a8f63301f5a9b267c63b4e97090…",0,"""call""",null
"""0x6b75d8af000000e20b7a7ddf000b…","""0x9bdc7dfd19b75b023e28bbb8e197…","""call""","""0x073760""","""0x3c8a7d8d00000000000000000000…",null,"""0x00""",null,null,"""0xb390d63aac03bbef75de888d16bd…",19999999,null,null,198965.0,"""0x0000000000000000000000000000…",5,"""0x0100000000000000010000000000…","""0x5a8f63301f5a9b267c63b4e97090…",0,"""call""",null


In [6]:
df.filter(pl.col("author").is_not_null()).collect()

from,to,call_type,gas,input,init,value,author,reward_type,block_hash,block_number,address,code,gas_used,output,subtraces,trace_address,transaction_hash,transaction_position,type,error
str,str,str,str,str,str,str,str,str,str,u64,str,str,f64,str,u64,str,str,u64,str,str
